In [47]:
import numpy as np
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.regularizers import l1_l2
from tensorflow import keras
import tensorflow as tf

from joblib import Parallel, delayed

np.set_printoptions(suppress=True, precision=6)
pd.set_option('display.float_format', '{:.6f}'.format)

In [48]:
df = pd.read_csv("data/otu_table_example.csv", index_col="Unnamed: 0").T

In [49]:
print(df.head())

             1050608  130468   3589405  355102   1081058  189592   354275   \
E000823.1.8        0        0        0        0        0        0        0   
E000823.2.6        0        0        0        0        0        0        0   
E000823.4.0        0        0        0        0        0        0        0   
E000823.5.0        0        0        0        0        0        0        0   
E000823.5.7        0        0        0        0        0        0        0   

             4327628  326749   183857   ...  317924   4294457  2655741  \
E000823.1.8        0        0        0  ...        0        0        0   
E000823.2.6        0        0        0  ...        0        0        0   
E000823.4.0        0        0        0  ...        0        0        0   
E000823.5.0        0        0        0  ...        0        0        0   
E000823.5.7        0        0        0  ...        0        0        0   

             858535   186092   299820   225846   4306049  366846   1124370  
E000823.1

In [50]:
# get rid of features that are way too sparse
zero_values_percentage_cutoff = .9

zero_counts = pd.Series([sum(df[col] == 0) for col in df.columns], index=df.columns)
zero_pcts = zero_counts / len(df)
populated_feats = zero_pcts[zero_pcts < zero_values_percentage_cutoff].index
df = df[populated_feats]

In [51]:
df

,189592,183857,4343580,198956,177319,505587,194104,193778,362389,2017729,...,368462,356360,183438,365385,178849,580008,296945,267718,354850,317924
E000823.1.8,0,0,2,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,7,0
E000823.2.6,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,2,0
E000823.4.0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,4,0,0,0,0,4,0
E000823.5.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,0,0,0,0,0
E000823.5.7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
E014086.30.4,1,0,4,0,13,0,6,0,1,0,...,0,0,0,1,0,446,0,5,7,1
E014086.32.4,0,0,0,0,44,0,16,0,0,3,...,0,4,0,0,0,851,0,3,50,1
E014086.33.5,0,0,3,0,11,0,11,0,0,2,...,0,3,0,0,0,664,0,3,46,1
E014086.34.4,1,0,1,0,9,0,9,0,0,0,...,0,1,0,0,0,180,0,3,28,0


In [52]:
def forward_rolling_average(series, window_size):
    
    original_idxs = series.index
    
    series = series.reset_index(drop=True)
    
    for idx in range(len(series) - window_size):
       
        next_vals = series[idx:idx+window_size]
        
        #print(idx, next_vals.tolist())
        
        series[idx] = next_vals.mean()
    
    series.index = original_idxs
    
    series = series.iloc[:-window_size]
            
    return series
    

In [53]:
window_size = 3
sequence = pd.Series([0,1,2,3,4,5,6,7,8,9,10])

forward_rolling_average(sequence, window_size)

0    1
1    2
2    3
3    4
4    5
5    6
6    7
7    8
dtype: int64

In [54]:
#def smooth_it_out(df, rolling_window=5):
#    # Define the window size for the rolling average
#    window_size = 5
#    
#    # Apply rolling mean to numeric columns
#    df = df.shift(-window_size + 1).rolling(window=window_size).mean()
#    
#    return df

In [55]:
def smooth_it_out(df, window_size=5, n_jobs=32):
    
    smooth_cols = Parallel(n_jobs=-n_jobs)(delayed(forward_rolling_average)(df[col], window_size) for col in df.columns)
    
    for idx, col in enumerate(df.columns):
        df[col] = smooth_cols[idx]
        
    df = df[:-window_size]
    
    return df

In [56]:
def feature_wise_scaling(df):
    for col in df.columns:
        _min_ = df[col].min()
        _max_ = df[col].max()
        
        df[col] = (df[col] - _min_) / (_max_ - _min_)
        
    return df      

In [57]:
def preprocess(feats_df, seq_length):
    num_features = len(feats_df.columns)
    
    X_sequences = []
    y_targets = []
    
    for i in range(len(feats_df) - seq_length):
        X_sequences.append(feats_df.iloc[i:i+seq_length])
        y_targets.append(feats_df.iloc[i+seq_length])
    
    X_sequences = np.array(X_sequences)
    y_targets = np.array(y_targets)
    
    X_sequences = X_sequences.reshape(-1, seq_length, num_features)
    
    return X_sequences, y_targets

In [58]:
def mae_ignore_zeros(y_true, y_pred, false_positives_penalty_factor=0.5):
    
    # Find indices where y_true is not zero
    non_zero_indices = tf.where(tf.not_equal(y_true, 0))
    
    # Gather the non-zero elements from y_true and y_pred using the indices
    y_true_non_zero = tf.gather_nd(y_true, non_zero_indices)
    y_pred_non_zero = tf.gather_nd(y_pred, non_zero_indices)
    
    y_true_non_zero = tf.cast(y_true_non_zero, tf.float64)
    y_pred_non_zero = tf.cast(y_pred_non_zero, tf.float64)
    
    # Calculate MAE on the non-zero elements
    mae_non_zero = tf.reduce_mean(tf.abs(y_pred_non_zero - y_true_non_zero))
    
    # Find indices where y_true is zero 
    zero_indices = tf.where(tf.equal(y_true, 0))
    
    # Gather the corresponding y_pred values
    y_pred_zero = tf.gather_nd(y_pred, zero_indices)
    
    y_pred_zero = tf.cast(y_pred_zero, tf.float64)
    
    # Calculate the average of false positives
    false_positives_avg = tf.reduce_mean(y_pred_zero)
    
    # Combine the MAE on non-zero elements with the average of false positives
    mae_ignore_zeros = mae_non_zero + (false_positives_avg * false_positives_penalty_factor)
    
    return mae_ignore_zeros

In [59]:
y_true = [0,0,10]
y_pred = [0,0,0]

keras.losses.mae(y_true, y_pred)

<tf.Tensor: shape=(), dtype=int32, numpy=3>

In [60]:
mae_ignore_zeros(y_true, y_pred)

<tf.Tensor: shape=(), dtype=float64, numpy=10.0>

In [61]:
df = smooth_it_out(df=df, window_size=5)
df

,189592,183857,4343580,198956,177319,505587,194104,193778,362389,2017729,...,368462,356360,183438,365385,178849,580008,296945,267718,354850,317924
E000823.1.8,0.000000,0.000000,0.600000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.200000,0.000000,0.000000,1.600000,0.000000,0.000000,0.000000,0.000000,2.600000,0.000000
E000823.2.6,0.000000,0.200000,0.200000,0.000000,0.000000,0.600000,0.000000,0.000000,0.400000,0.000000,...,0.200000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,1.200000,0.000000
E000823.4.0,0.000000,0.200000,0.000000,0.000000,0.000000,0.600000,0.000000,0.000000,0.400000,0.000000,...,0.200000,0.000000,0.000000,1.800000,0.000000,0.000000,0.000000,0.000000,0.800000,0.000000
E000823.5.0,0.000000,0.400000,0.200000,0.000000,0.000000,0.600000,0.000000,0.000000,0.400000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
E000823.5.7,0.200000,0.600000,0.200000,0.000000,2.600000,0.600000,8.400000,0.000000,1.000000,0.200000,...,0.600000,0.000000,0.800000,13.800000,0.000000,0.000000,0.000000,0.000000,0.800000,0.400000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
E014086.22.5,1.200000,0.000000,1.600000,0.000000,3.200000,0.000000,8.000000,0.000000,0.000000,0.800000,...,0.000000,0.600000,0.000000,29.200000,0.000000,397.600000,0.000000,8.200000,17.400000,0.600000
E014086.23.4,1.400000,0.000000,2.400000,0.000000,5.200000,0.000000,7.800000,0.000000,0.200000,0.800000,...,0.000000,0.400000,0.000000,29.400000,0.000000,486.800000,0.000000,8.800000,12.600000,0.800000
E014086.24.5,1.400000,0.000000,1.400000,0.000000,13.800000,0.000000,11.000000,0.000000,0.200000,1.400000,...,0.000000,1.200000,0.000000,29.400000,0.000000,657.000000,0.000000,9.400000,19.600000,0.600000
E014086.26.4,0.200000,0.000000,2.000000,0.000000,15.600000,0.000000,10.200000,0.000000,0.200000,1.600000,...,0.000000,1.800000,0.000000,0.200000,0.000000,673.800000,0.000000,6.200000,22.000000,0.800000


In [62]:
df = feature_wise_scaling(df)
df

/tmp/ipykernel_79223/2433063030.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = (df[col] - _min_) / (_max_ - _min_)


,189592,183857,4343580,198956,177319,505587,194104,193778,362389,2017729,...,368462,356360,183438,365385,178849,580008,296945,267718,354850,317924
E000823.1.8,0.000000,0.000000,0.085714,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.007407,0.000000,0.000000,0.000678,0.000000,0.000000,0.000000,0.000000,0.043189,0.000000
E000823.2.6,0.000000,0.055556,0.028571,0.000000,0.000000,0.007979,0.000000,0.000000,0.048780,0.000000,...,0.007407,0.000000,0.000000,0.000848,0.000000,0.000000,0.000000,0.000000,0.019934,0.000000
E000823.4.0,0.000000,0.055556,0.000000,0.000000,0.000000,0.007979,0.000000,0.000000,0.048780,0.000000,...,0.007407,0.000000,0.000000,0.000763,0.000000,0.000000,0.000000,0.000000,0.013289,0.000000
E000823.5.0,0.000000,0.111111,0.028571,0.000000,0.000000,0.007979,0.000000,0.000000,0.048780,0.000000,...,0.000000,0.000000,0.000000,0.000424,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
E000823.5.7,0.002538,0.166667,0.028571,0.000000,0.151163,0.007979,0.330709,0.000000,0.121951,0.076923,...,0.022222,0.000000,0.181818,0.005852,0.000000,0.000000,0.000000,0.000000,0.013289,0.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
E014086.22.5,0.015228,0.000000,0.228571,0.000000,0.186047,0.000000,0.314961,0.000000,0.000000,0.307692,...,0.000000,0.083333,0.000000,0.012382,0.000000,0.141374,0.000000,0.310606,0.289037,0.250000
E014086.23.4,0.017766,0.000000,0.342857,0.000000,0.302326,0.000000,0.307087,0.000000,0.024390,0.307692,...,0.000000,0.055556,0.000000,0.012467,0.000000,0.173091,0.000000,0.333333,0.209302,0.333333
E014086.24.5,0.017766,0.000000,0.200000,0.000000,0.802326,0.000000,0.433071,0.000000,0.024390,0.538462,...,0.000000,0.166667,0.000000,0.012467,0.000000,0.233608,0.000000,0.356061,0.325581,0.250000
E014086.26.4,0.002538,0.000000,0.285714,0.000000,0.906977,0.000000,0.401575,0.000000,0.024390,0.615385,...,0.000000,0.250000,0.000000,0.000085,0.000000,0.239582,0.000000,0.234848,0.365449,0.333333


In [63]:
df.describe()

,189592,183857,4343580,198956,177319,505587,194104,193778,362389,2017729,...,368462,356360,183438,365385,178849,580008,296945,267718,354850,317924
count,516.000000,516.000000,516.000000,516.000000,516.000000,516.000000,516.000000,516.000000,516.000000,516.000000,...,516.000000,516.000000,516.000000,516.000000,516.000000,516.000000,516.000000,516.000000,516.000000,516.000000
mean,0.073806,0.098514,0.182281,0.109496,0.083153,0.017138,0.134865,0.031603,0.190442,0.099583,...,0.029113,0.100883,0.054175,0.050919,0.077726,0.064489,0.050222,0.105092,0.234110,0.103198
std,0.137929,0.126745,0.167185,0.174509,0.144020,0.099215,0.218964,0.117428,0.196432,0.164082,...,0.103716,0.138311,0.122820,0.135822,0.184834,0.130804,0.119696,0.172336,0.214674,0.173783
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.057143,0.000000,0.000000,0.000000,0.000000,0.000000,0.024390,0.000000,...,0.000000,0.000000,0.000000,0.000085,0.000000,0.000000,0.000000,0.000000,0.053156,0.000000
50%,0.010152,0.055556,0.142857,0.000000,0.023256,0.000000,0.015748,0.000000,0.146341,0.000000,...,0.000000,0.055556,0.000000,0.002460,0.000000,0.000213,0.000000,0.030303,0.169435,0.000000
75%,0.076142,0.166667,0.257143,0.200000,0.104651,0.007979,0.204724,0.000000,0.292683,0.153846,...,0.014815,0.138889,0.045455,0.042299,0.021277,0.081301,0.038005,0.138258,0.372924,0.166667
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [65]:
seq_length = 50
cutoff = 411

X_sequences, y_targets = preprocess(feats_df=df, seq_length=seq_length)

In [66]:
X_sequences_train = X_sequences[:cutoff]
y_targets_train = y_targets[:cutoff]

X_sequences_test = X_sequences[cutoff:]
y_targets_test = y_targets[cutoff:]

print(f"Length of data: {len(df)}")
print(f"Total sequences: {len(X_sequences)}")
print(f"Train sequences: {len(X_sequences_train)}")
print(f"Test sequences: {len(X_sequences_test)}")

Length of data: 516
Total sequences: 466
Train sequences: 411
Test sequences: 55


In [67]:
n_feats = len(df.columns)

In [72]:
reg = 1e-12
loss=mae_ignore_zeros

model = Sequential()
model.add(keras.Input(shape=(seq_length,n_feats)))
model.add(keras.layers.LSTM(1024, return_sequences=False, activation='relu'))
model.add(keras.layers.Dense(1024, activation="relu", kernel_regularizer=l1_l2(reg)))
model.add(keras.layers.Dense(n_feats, activation="relu", kernel_regularizer=l1_l2(reg)))

model.compile(optimizer="Adam", loss=loss, metrics=["mae", "mape"])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 1024)              10096640  
                                                                 
 dense_4 (Dense)             (None, 1024)              1049600   
                                                                 
 dense_5 (Dense)             (None, 1440)              1476000   
                                                                 
Total params: 12622240 (48.15 MB)
Trainable params: 12622240 (48.15 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [73]:
model.fit(x=X_sequences_train, y=y_targets_train, validation_split=0.1, epochs=10)

Epoch 1/10
12/12 [==============================] - 2s 86ms/step - loss: 16.5061 - mae: 0.0853 - mape: 7369157.0000 - val_loss: 10.3418 - val_mae: 0.0740 - val_mape: 5930173.0000
Epoch 2/10
12/12 [==============================] - 1s 64ms/step - loss: 7.2114 - mae: 0.0779 - mape: 7098719.0000 - val_loss: 4.1097 - val_mae: 0.0717 - val_mape: 6552893.5000
Epoch 3/10
12/12 [==============================] - 1s 67ms/step - loss: 2.7208 - mae: 0.0753 - mape: 7226336.0000 - val_loss: 1.4260 - val_mae: 0.0711 - val_mape: 7226200.5000
Epoch 4/10
12/12 [==============================] - 1s 65ms/step - loss: 0.9183 - mae: 0.0734 - mape: 7414288.5000 - val_loss: 0.4765 - val_mae: 0.0709 - val_mape: 6733617.5000
Epoch 5/10
12/12 [==============================] - 1s 64ms/step - loss: 0.3259 - mae: 0.0720 - mape: 7068530.5000 - val_loss: 0.2067 - val_mae: 0.0711 - val_mape: 8857123.0000
Epoch 6/10
12/12 [==============================] - 1s 64ms/step - loss: 0.1689 - mae: 0.0704 - mape: 7356337.500

In [ ]:
y_pred = model.predict(X_sequences_test)

In [ ]:
y_test_df = pd.DataFrame(y_targets_test)
y_pred_df = pd.DataFrame(y_pred)

In [ ]:
y_test_df

In [ ]:
y_pred_df

In [ ]:
target_taxa = 3

sns.lineplot(y_test_df[target_taxa])
plt.title(f"True sequence for taxa_idx {target_taxa}")
plt.show()

sns.lineplot(y_pred_df[target_taxa])
plt.title(f"Pred sequence for taxa_idx {target_taxa}")
plt.show()

In [ ]:
def calculate_errors(y_pred_df, y_test_df):
    
    errors_df = []
    for col in y_pred_df.columns:
        errors = abs((y_test_df[col] - y_pred_df[col] ) / (y_test_df[col] + 1e-10))
        errors_df.append(errors)
        
    errors_df = pd.concat(errors_df, axis=1)
    
    return errors_df

In [ ]:
errors_df = calculate_errors(y_pred_df, y_test_df)
errors_df.describe()